## Finding the most profitable towns in a CCAA given a max price

In [ ]:
import pandas as pd
import math
from bs4 import BeautifulSoup
import requests
import nums_from_string as nfs
import numpy as np
from re import search
from random import randint
from time import sleep
import openpyxl
import xlrd
import lxml
from datetime import datetime, timedelta
from collections import Counter
import re
pd.options.mode.chained_assignment = None
import seaborn as sns
from functions import *

#### Importing geo data from csv file

This enables us to query for a list of towns given an autonomous community

In [ ]:
geo_data = pd.read_csv('/Users/ignaciolorenzoqueralt/Documents/Ironhack/Final Project/data/province-town-n_props/geo-data_2021.10.23.csv')

#### Get input from user

In [ ]:
ccaa_lst = geo_data['ccaa'].unique().tolist()
ccaa = input("ccaa: ")
while ccaa not in ccaa_lst:
    ccaa = input("There was no match between your input and our ccaa, try again: ")

In [ ]:
num_properties_per_town = ""
while num_properties_per_town == "":
    try:
        num_properties_per_town = int(input("Minimum number of properties per town: "))
    except: 
        num_properties_per_town = ""

In [5]:
filtered_df = geo_data[(geo_data['ccaa'] == ccaa) & (geo_data['n_properties'] > num_properties_per_town)]

In [6]:
max_price = ""
while max_price == "":
    try:
        max_price = int(input("maximum price: "))
    except: 
        max_price = ""

maximum price:  90000


#### List of towns that we need to extract data from

In [7]:
towns = filtered_df[filtered_df['ccaa'] == ccaa].town.to_list()

#### Extracting data from each town

In [8]:
property = "viviendas"
municipio = "hospitalet_de_llobregat"
ascensor = "-ascensor"

habitaciones = "hab="+"1"
baños = "&"+"ban="+"1"
maximum_price = "pmax="+str(max_price)
metros = "m2="+"50"

In [9]:
#url = "https://www.habitaclia.com/"+property+ascensor+"-"+municipio+".htm?"+habitaciones+baños+maximum_price

In [10]:
# Creation of the df to which we will append the properties of the selected towns.
name = []
town = []
area = []
neighborhood = []
geo_town = []
features = []
m2 = []
n_rooms = []
n_bath = []
price_m2 = []
description = []
price = []
opportunity = []
price_reduction = []
opportunity = []
last_update = []
url = []
lift = []

x = min(len(name), len(town), len(area), len(neighborhood), len(geo_town), len(description), len(price), len(last_update), len(url), len(lift))
dct = {'name': name[:x], 'town': town[:x], 'area': area[:x], 'neighborhood': neighborhood[:x], 'geo_town':geo_town[:x],'m2': m2[:x], 'n_rooms': n_rooms[:x], 'n_bath': n_bath[:x], 'price_m2': price_m2[:x] ,'price': price[:x], 'price_reduction': price_reduction[:x], 'opportunity':opportunity[:x], 'last_update': last_update[:x],  'description': description[:x], 'url':url[:x], 'lift': lift[:x]}
df = pd.DataFrame.from_dict(dct)

# Pulling properties from each town in the previously defined list:

for t in towns:
    
    # Getting the number of properties for that town to see how many pages do we need to scrape.
    url = "https://www.habitaclia.com/"+property+"-"+t+".htm?"+maximum_price
    r = requests.get(url)
    r.status_code
    soup = BeautifulSoup(r.content, 'html.parser')
    
    try:
        total_results = int(soup.find('h2', attrs={'class': 'f-right'}).find('span').get_text().replace(".",""))
        pages = range(int(math.floor(total_results/16))+1)
        properties = []

        # Adding all the properties listed in each page to the list.
        for p in pages:
            sleep(randint(1,3))
            url = "https://www.habitaclia.com/"+property+"-"+t+"-"+str(p)+".htm?"+maximum_price
            try:
                r = requests.get(url)
                soup = BeautifulSoup(r.content, 'html.parser')
                properties += soup.find_all('div', attrs={'class': 'list-item-info'})
                del properties[-1] # Last item is an ad
            except:
                print('Error on page', p)
                print('town: ', t, " page: ", p, "url: ", url)

        # Creating a list for each piece of information I want to extract from each property.
        name = []
        town = []
        area = []
        neighborhood = []
        geo_town = []
        features = []
        m2 = []
        n_rooms = []
        n_bath = []
        price_m2 = []
        description = []
        price = []
        opportunity = []
        price_reduction = []
        opportunity = []
        last_update = []
        url = []
        lift = []

        # Getting the information from each property.
        for i,properties in enumerate(properties):
            #print(i)
            # Each feature is set as empty prior to being defined. This way we avoid errors when a feature is not available for a certain property.
            name_temp = ""
            town_temp = ""
            area_temp = ""
            neighborhood_temp = ""
            geo_town_temp = ""
            m2_temp = ""
            n_rooms_temp = ""
            n_bath_temp = ""
            price_m2_temp = ""
            price_temp = ""
            opportunity_temp = ""
            price_reduction_temp = ""
            description_temp = ""
            last_update_temp = ""
            url_temp = ""
            lift_temp = ""

            # other_location enables us to differ between listed properties vs suggested properties, which appear when there are very few properties for one town. We want to avoid them as they are nearby properties not belonging to our target town.
            other_location = properties.find('span', attrs={'class': 'ady-relationship'})
            if other_location is None:
                other_locations_properties = ""
            else: 
                #print(i)
                other_locations_properties = other_location.get_text(strip=True).find('Se encuentra en')

            # Now I am skipping all the properties that are suggested so as to not append them to the df.
            if other_locations_properties == 0:
                pass
            else:
                try:
                    # Extracting the features of a property and saving them in a temporary variable.
                    name_temp = properties.find('h3', attrs={'class': 'list-item-title'}).get_text(strip=True)
                    town_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).split("-",1)[0].strip().replace("Ver mapa","")
                    area_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[0]
                    neighborhood_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[1].strip()
                    geo_town_temp = t
                    m2_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[0])[0] 
                    n_rooms_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[1])[0]
                    n_bath_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[2])[0] 
                    price_m2_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[3])[0] 
                    raw_price_temp = properties.find('article', attrs={'class': 'list-item-price'}).get_text()
                    if search("Oportunidad", raw_price_temp):
                        if search("ha bajado", raw_price_temp): 
                            price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            price_reduction_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]
                            opportunity_temp = "yes"
                        else:
                            price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            price_reduction_temp = "0"
                            opportunity_temp = "yes"
                    elif search("ha bajado", raw_price_temp):
                        price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        price_reduction_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]      
                        opportunity_temp = "no"
                    else: 
                        price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        price_reduction_temp = "0"
                        opportunity_temp = "no"
                    description_temp = properties.find('p', attrs={'class': 'list-item-description'}).get_text(strip=True)
                    last_update_temp = nfs.get_nums(properties.find('span', attrs={'class': 'list-item-date'}).get_text(strip=True))
                    url_temp = properties.find('h3', attrs={'class': 'list-item-title'}).find('a').get('href')
                    lift_temp = ""
                    
                    # Appending temporary variables features to their corresponding list.
                    name.append(name_temp)
                    town.append(town_temp)
                    area.append(area_temp)
                    neighborhood.append(neighborhood_temp)
                    geo_town.append(geo_town_temp)
                    m2.append(m2_temp)
                    n_rooms.append(n_rooms_temp)
                    n_bath.append(n_bath_temp)
                    price_m2.append(price_m2_temp)
                    price.append(price_temp)
                    opportunity.append(opportunity_temp)
                    price_reduction.append(price_reduction_temp)
                    description.append(description_temp)
                    last_update.append(last_update_temp)
                    url.append(url_temp)
                    lift.append(lift_temp)

                except:
                    # In case we may encounter an error, we print the features of each property to find the bug.
                    '''
                    print('------------------------------------')
                    print('nombre: ', name_temp)
                    print('town_temp: ', town_temp)
                    print('area_temp: ', area_temp)
                    print('m2_temp: ', m2_temp)
                    print('n_rooms_temp: ', n_rooms_temp)
                    print('n_bath_temp: ', n_bath_temp)
                    print('price_m2_temp: ', price_m2_temp)
                    print('price_temp: ', price_temp)
                    print('opportunity_temp: ', opportunity_temp)
                    print('price_reduction_temp: ', price_reduction_temp)
                    print('description_temp: ', description_temp)
                    print('last_update_temp: ', last_update_temp)
                    print('url_temp: ', url_temp)
                    print('------------------------------------')
                    '''
        
        x_town = min(len(name), len(town), len(area), len(neighborhood), len(geo_town), len(description), len(price), len(last_update), len(url), len(lift))
        dct_town = {'name': name[:x_town], 'town': town[:x_town], 'area': area[:x_town], 'neighborhood': neighborhood[:x_town], 'geo_town':geo_town[:x_town],'m2': m2[:x_town], 'n_rooms': n_rooms[:x_town], 'n_bath': n_bath[:x_town], 'price_m2': price_m2[:x_town] ,'price': price[:x_town], 'price_reduction': price_reduction[:x_town], 'opportunity':opportunity[:x_town], 'last_update': last_update[:x_town],  'description': description[:x_town], 'url':url[:x_town], 'lift': lift[:x_town]}
        df_town = pd.DataFrame.from_dict(dct_town)
        df = df.append(df_town, ignore_index = True)
        
        #Cleaning the final df
        #df = df[~df['description'].str.contains('nuda|sin cedula|sin cédula')]
    except:
        pass

#### Output from the web scrapping

In [11]:
df.shape

(6863, 16)

In [12]:
df.head(3)

,name,town,area,neighborhood,geo_town,m2,n_rooms,n_bath,price_m2,price,price_reduction,opportunity,last_update,description,url,lift
0,"Piso en Plaça sa boada, 35. Oportunidad en e...",Arenys de Mar,Arenys de Mar,Centre,arenys_de_mar,50.0,3.0,1.0,1.898,94900.0,100,yes,[],Tenemos en exclusiva esta vivienda visitas con...,https://www.habitaclia.com/comprar-piso-oportu...,
1,Apartamento Carrer avall (d´). Apartamento en...,Arenys de Mar,Arenys de Mar,Centre,arenys_de_mar,41.0,1.0,1.0,2.171,89000.0,0,no,[3],Disseny i habitatge les presenta este fantásti...,https://www.habitaclia.com/comprar-apartamento...,
2,Piso Carrer camí de la pietat. Piso economico...,Arenys de Mar,Arenys de Mar,Centre,arenys_de_mar,50.0,3.0,1.0,1.774,88700.0,0,no,[3],"Piso de 50 m2, con comedor de 15 m2, cocina in...",https://www.habitaclia.com/comprar-piso-econom...,


#### Cleaning the df

In [13]:
df1 = df.copy()

In [14]:
df1 = df1.merge(geo_data, left_on='geo_town', right_on='town', how='left')
df1 = df1.drop(["town_x", "town_y", "n_properties"], axis=1)
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]

In [15]:
numericals = ['m2', 'n_rooms', 'n_bath', 'price_m2', 'price', 'price_reduction', 'last_update']

def anytype_to_numerical(df, columns = []):
    for c in columns:
        if df[c].dtypes == 'float64':
            df[c] = df[c].astype(int)

def clean_last_update():
    for i,n in enumerate(df1['last_update']):
        try:
            df1['last_update'][i] = df1['last_update'][i][0]
        except:
            df1['last_update'][i] = '0'

In [16]:
anytype_to_numerical(df1, columns = numericals)
clean_last_update()

In [17]:
df1= df1[df1['price']<int(max_price)]

In [18]:
df1['lift'] = None

In [19]:
df1.shape

(4497, 17)

#### Getting info about lifts

In [ ]:
dct = {'name': [],
       'town':[],
       'area': [],
       'neighborhood': [],
       'geo_town': [],
       'm2': [],
       'n_rooms': [],
       'n_bath': [],
       'price_m2': [],
       'description': [],
       'price': [],
       'opportunity': [],
       'price_reduction': [],
       'last_update': [],
       'url': [],
       'lift': []}
# Data Scraping begins
for t in towns:
    url = "https://www.habitaclia.com/"+property+"-"+"viviendas-ascensor"+"-"+t+".htm?"+maximum_price
    r = requests.get(url)
    r.status_code
    soup = BeautifulSoup(r.content, 'html.parser')
    try:
        # Getting the number of properties for that town to see how many pages do we need to scrape.
        total_results = int(soup.find('h2', attrs={'class': 'f-right'}).find('span').get_text().replace(".",""))
        pages = range(int(math.floor(total_results/16))+1)
        properties = []
        # Adding all the properties listed in each page to the list.
        for p in pages:
            sleep(randint(2,3))
            url = "https://www.habitaclia.com/"+property+"-"+"viviendas-ascensor"+"-"+t+"-"+str(p)+".htm?"+maximum_price
            try:
                r = requests.get(url)
                soup = BeautifulSoup(r.content, 'html.parser')
                properties += soup.find_all('div', attrs={'class': 'list-item-info'})
                del properties[-1] # Last item is an ad
            except:
                print('Error on page', p)
                print('town: ', t, " page: ", p, "url: ", url)
                
        # Getting the information from each property.
        for i,properties in enumerate(properties):  
            dct_2 = {}
            dct_3 = {}
            for k, v in dct.items():
                k_temp = str(k)+'_temp'
                dct_2[k] = k_temp
            for k,v in dct_2.items():
                dct_3[v] = ""
            
            # other_location enables us to differ between listed properties vs suggested properties, which appear when there are very few properties for one town. We want to avoid them as they are nearby properties not belonging to our target town.
            other_location = properties.find('span', attrs={'class': 'ady-relationship'})
            if other_location is None:
                other_locations_properties = ""
            else: 
                other_locations_properties = other_location.get_text(strip=True).find('Se encuentra en')

            # Now I am skipping all the properties that are suggested so as to not append them to the df.
            if other_locations_properties == 0:
                pass
            else:
                try:
                    # Extracting the features of a property and saving them in a temporary variable.
                    dct_3['name_temp'] = properties.find('h3', attrs={'class': 'list-item-title'}).get_text(strip=True)
                    dct_3['town_temp'] = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).split("-",1)[0].strip().replace("Ver mapa","")
                    dct_3['area_temp'] = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[0]
                    dct_3['neighborhood_temp'] = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[1].strip()
                    dct_3['geo_town_temp'] = t
                    dct_3['m2_temp'] = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[0])[0] 
                    dct_3['n_rooms_temp'] = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[1])[0]
                    dct_3['n_bath_temp'] = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[2])[0] 
                    dct_3['price_m2_temp'] = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[3])[0] 
                    dct_3['raw_price_temp'] = properties.find('article', attrs={'class': 'list-item-price'}).get_text()
                    
                    if search("Oportunidad", dct_3['raw_price_temp']):
                        if search("ha bajado", dct_3['raw_price_temp']): 
                            dct_3['price_temp'] = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            dct_3['price_reduction_temp'] = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]
                            dct_3['opportunity_temp'] = "yes"
                        else:
                            dct_3['price_temp'] = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            dct_3['price_reduction_temp'] = "0"
                            dct_3['opportunity_temp'] = "yes"
                    elif search("ha bajado", dct_3['raw_price_temp']):
                        dct_3['price_temp'] = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        dct_3['price_reduction_temp'] = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]      
                        dct_3['opportunity_temp'] = "no"
                    else: 
                        dct_3['price_temp'] = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        dct_3['price_reduction_temp'] = "0"
                        dct_3['opportunity_temp'] = "no"
                    dct_3['description_temp'] = properties.find('p', attrs={'class': 'list-item-description'}).get_text(strip=True)
                    dct_3['last_update_temp'] = nfs.get_nums(properties.find('span', attrs={'class': 'list-item-date'}).get_text(strip=True))
                    dct_3['url_temp'] = properties.find('h3', attrs={'class': 'list-item-title'}).find('a').get('href')
                    dct_3['lift_temp'] = 'yes'
                    # Appending temporary variables features to their corresponding dictionary.
                    for k,v in dct.items():
                        temp = k+'_temp'
                        dct[k].append(dct_3[temp])
                    
                except:
                    print('error')
        
        x = min(map(len, dct.values()))
        df = pd.DataFrame({k:v[:x] for k,v in dct.items()})
        
    except:
        print("no properties found at: ", url)

In [21]:
df1_lift = df_lift.copy()

In [22]:
df1_lift.shape

(2327, 16)

In [23]:
df1_lift = df1_lift.merge(geo_data, left_on='geo_town', right_on='town', how='left')
df1_lift = df1_lift.drop(["town_x", "town_y", "n_properties"], axis=1)
df1_lift = df1_lift.loc[:, ~df1_lift.columns.str.contains('^Unnamed')]

In [24]:
numericals = ['m2', 'n_rooms', 'n_bath', 'price_m2', 'price', 'price_reduction', 'last_update']

def anytype_to_numerical(df, columns = []):
    for c in columns:
        if df[c].dtypes == 'float64':
            df[c] = df[c].astype(int)

def clean_last_update():
    for i,n in enumerate(df1_lift['last_update']):
        try:
            df1_lift['last_update'][i] = df1_lift['last_update'][i][0]
        except:
            df1_lift['last_update'][i] = '0'

In [25]:
anytype_to_numerical(df1_lift, columns = numericals)
clean_last_update()

In [26]:
df1_lift= df1_lift[df1_lift['price']<int(max_price)]

In [27]:
df1_lift.shape

(1335, 17)

In [28]:
list_names_with_lift = list(df1_lift['name'])

In [29]:
final_df = df1.combine_first(df1_lift)
final_df.reset_index(drop=True, inplace=True)

In [30]:
final_df.shape

(5126, 17)

#### Data about the floor of each property

In [31]:
final_df.insert(1, 'floor', '')

In [32]:
final_df.head(3)

,name,floor,area,neighborhood,geo_town,m2,n_rooms,n_bath,price_m2,price,price_reduction,opportunity,last_update,description,url,lift,province,ccaa
0,Apartamento Carrer avall (d´). Apartamento en...,,Arenys de Mar,Centre,arenys_de_mar,41,1,1,2,89000,0,no,3,Disseny i habitatge les presenta este fantásti...,https://www.habitaclia.com/comprar-apartamento...,NaN,barcelona,cataluña
1,Piso Carrer camí de la pietat. Piso economico...,,Arenys de Mar,Centre,arenys_de_mar,50,3,1,1,88700,0,no,3,"Piso de 50 m2, con comedor de 15 m2, cocina in...",https://www.habitaclia.com/comprar-piso-econom...,yes,barcelona,cataluña
2,Piso Carrer de la independència. Oportunidad ...,,Badalona,Bufalà,badalona,77,1,1,1,79000,0,no,0,CÓD. 1689 ¡¡¡OPORTUNIDAD!! LOFT en venta de 77...,https://www.habitaclia.com/comprar-piso-oportu...,yes,barcelona,cataluña


In [33]:
lst_underground_floor = ['sótano', 'sotano', 'semi sotano', 'semi sótano', 'semisotano', 'semisótano']
lst_ground_floor = ['bajo', 'bajos']
lst_first_floor = ['primera planta', 'primer piso', 'primero']
lst_second_floor = ['segunda planta', 'segundo piso', 'segundo']
lst_third_floor = ['tercera planta', 'tercer piso', 'tercero']
lst_fourth_floor = ['cuarta planta', 'cuarto piso']
lst_fifth_floor = ['quinta planta', 'quinto piso', 'quinto']
lst_sixth_floor = ['sexta planta', 'sexto piso', 'sexto']
lst_seventh_floor = ['séptima planta', 'séptimo piso', 'séptimo', 'septima planta', 'septimo piso', 'septimo']
lst_eighth_floor = ['octava planta', 'octavo piso', 'octavo']
lst_ninth_floor = ['novena planta', 'noveno piso', 'noveno']
lst_tenth_floor = ['décima planta', 'décimo piso', 'décimo', 'décima planta', 'décimo piso', 'décimo']

list_floors = [lst_underground_floor, 
               lst_ground_floor,
               lst_first_floor,
               lst_second_floor,
               lst_third_floor,
               lst_fourth_floor,
               lst_fifth_floor,
               lst_sixth_floor,
               lst_seventh_floor,
               lst_eighth_floor,
               lst_ninth_floor,
               lst_tenth_floor]

Floor data in the description

In [34]:
for l in list_floors:
    find_floor(final_df, 
               column = 'description', 
               key_words = l)

Floor data from urls

In [35]:
get_lift_from_property_url(final_df)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
2000
2100
2200
2300
2400
2500
2700
2800
2900
3000
3100
3300
3400
3500
3700
3800
4000
4100
4200
4300
4400
4500
4600
4700
4800
5000
5100


In [36]:
final_df1 = final_df.copy()

In [37]:
clean_floors(final_df1)

#### Saving the scraped df

In [38]:
today = datetime.now().strftime('%Y.%m.%d')
final_df1.to_csv(path_or_buf = '/Users/ignaciolorenzoqueralt/Documents/Ironhack/Final Project/properties/sale/'+today+'_'+ccaa+'_'+str(max_price)+'_'+str(num_properties_per_town)+'.csv')